In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from preprop import *
import os
import re

import emoji
from soynlp.normalizer import repeat_normalize

In [2]:
data_dir = ['Unlabeled_data1', 'Unlabeled_data2', 'Unlabeled_data3']
dfall = []
for d in data_dir :
    d = 'unlabel_data/' + d
    files = [f for f in os.listdir(d) if f.endswith('xls') or f.endswith('xlsx')]
    for f in files :
        x1 = pd.ExcelFile( d + '/' + f)
#         print(x1.sheet_names)
        for sheet in x1.sheet_names :
            df = x1.parse(sheet)
            print(df.columns)
            print(df.shape)
            dfall.append(df)
            
df = pd.concat(dfall)      
print(df.shape)
df.reset_index(drop=True, inplace=True)
df.to_csv('files/unlabel.csv', index=None)


Index(['환자ID', '보호자명', '환자명', '품종코드', '품종', '생일', '성별코드', '성별', '진단일', '진단분류',
       '진단명', 'SE기록일', 'SE'],
      dtype='object')
(38557, 13)
Index(['환자ID', '보호자명', '환자명', '품종코드', '품종', '생일', '성별코드', '성별', '진단일', '진단분류',
       '진단명', 'SE기록일', 'SE'],
      dtype='object')
(56346, 13)
Index(['환자ID', '보호자명', '환자명', '품종코드', '품종', '생일', '성별코드', '성별', '진단일', '진단분류',
       '진단명', 'SE기록일', 'SE'],
      dtype='object')
(35266, 13)
Index(['환자ID', '보호자명', '환자명', '품종코드', '품종', '생일', '성별코드', '성별', '진단일', '진단분류',
       '진단명', 'SE기록일', 'SE'],
      dtype='object')
(11965, 13)
Index(['환자ID', '보호자명', '환자명', '품종코드', '품종', '생일', '성별코드', '성별', '진단일', '진단분류',
       '진단명', 'SE기록일', 'SE'],
      dtype='object')
(62317, 13)
Index(['환자ID', '보호자명', '환자명', '품종코드', '품종', '생일', '성별코드', '성별', '진단일', '진단분류',
       '진단명', 'SE기록일', 'SE'],
      dtype='object')
(64478, 13)
Index(['환자ID', '보호자명', '환자명', '품종코드', '품종', '생일', '성별코드', '성별', '진단일', '진단분류',
       '진단명', 'SE기록일', 'SE'],
      dtype='object')
(39577, 13)

In [89]:
# df.to_csv('files/unlabel.csv', index=None)
df = pd.read_csv('files/unlabel.csv')
df = df[['진단명', 'SE']]
df.shape

(981339, 2)

In [90]:
x2 = pd.ExcelFile('unlabel_data/PMS_unlabel_data.xlsx')
for sheet in x2.sheet_names :
    print(sheet)
    df1 = x2.parse(sheet)
    print(df1.columns)
df1 = x2.parse('Sheet2')


Sheet2
Index(['Sheet 1', 'ASSESSMENT_NAME', 'CHART_ID', 'CHART_MEMO', 'CHART_DATE',
       'COMPANY_NAME'],
      dtype='object')
Sheet1
Index([], dtype='object')


In [91]:
df1 = df1[['ASSESSMENT_NAME', 'CHART_MEMO']]
df1.columns = ['진단명', 'SE']
df1.shape

(106750, 2)

In [92]:
df = pd.concat([df, df1])
df.reset_index(drop=True, inplace=True)
df.shape

(1088089, 2)

In [67]:
num_cls = 10

In [68]:
# for i in range(0,20) :
#     print(df['진단명'].value_counts()[i*num_cls:(i+1)*num_cls]) 
#     print(" ************************************************ ", df['진단명'].value_counts()[i*num_cls:(i+1)*num_cls].sum()) 

In [96]:
df['진단명'].value_counts()[0:10]

Pancreatitis                                4464
RESPIRATORY DISTRESS SYNDROME, ACUTE        3194
Otitis Externa                              3149
Anti-inflamation                            2844
Malassezia Dermatitis                       2374
Vomiting, Acute                             2244
Pulmonary Edema                             2104
Dyspnea and Tachypnea                       2012
Chronic Renal Disease (CKD) - IRIS stage    1947
dermatits-bacterial                         1846
Name: 진단명, dtype: int64

In [97]:
df.dropna(subset=['SE'], inplace=True)
df.shape

(1065829, 2)

In [98]:
df.drop_duplicates(subset=['SE'], inplace=True)
df.shape

(872491, 2)

In [99]:
df.dropna(subset=['진단명'], inplace=True)
df.shape

(135101, 2)

In [100]:
df['len'] = df['SE'].apply(lambda x : len(str(x)))
df = df[df['len'] > 40]
df.shape

(113044, 3)

In [101]:
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

In [102]:
def clean(x):
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

In [103]:
df.SE = df.SE.apply(lambda x : clean(str(x)))

In [104]:
def clean1(text) :
    text = phone_number_filter(text)
    text = url_filter(text)
    text = price_filter(text)
    text = date_filter(text)
    text = time_filter(text)
    text = remove_name(text)
    text = re.sub('[ ][0-9]+[ ]','', text )
    text = re.sub('[\s]+[a-z][\s]+', ' ', text )
    text = re.sub('[.]+', '.', text )
    text = re.sub('[-]+', '-', text )
    text = re.sub('[ ]+', ' ', text )    
    return text

In [105]:
df.SE = df.SE.apply(lambda x : preprocess(str(x)))
# df.SE = df.SE.apply(lambda x : clean1(str(x)))
df['len'] = df['SE'].apply(lambda x : len(str(x)))

In [106]:
df = df[df['len'] > 40]
df.shape

(109218, 3)

In [114]:
df['진단명'].value_counts()[0:400]

Anti-inflamation                        1571
dermatits-bacterial                     1294
Pancreatitis                            1184
Pulmonary Edema                         1182
Malassezia Dermatitis                   1009
                                        ... 
Intervertebral Disc Disease (IVDD)()      48
파보바이러스 감염 (Parvoviral Infection)          48
폐수종(Pulmonary Edema)                      48
Clostridial Enterotoxicosis               48
Demodicosis                               48
Name: 진단명, Length: 400, dtype: int64

In [109]:
import pandas as pd
import numpy as np
import os
import re
import sys
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [115]:
label_dict = df['진단명'].value_counts()[0:400].to_dict()


In [116]:
df['진단명1'] = df['진단명'].apply(lambda x : x if str(x) in label_dict.keys() else np.nan)    

In [117]:
df.dropna(subset=['진단명1'], inplace=True)
df.shape

(62431, 4)

In [118]:
df['진단명1'].value_counts()[0:400]

Anti-inflamation                        1571
dermatits-bacterial                     1294
Pancreatitis                            1184
Pulmonary Edema                         1182
Malassezia Dermatitis                   1009
                                        ... 
각막궤양                                      48
Intervertebral Disc Disease (IVDD)()      48
파보바이러스 감염 (Parvoviral Infection)          48
Demodicosis                               48
Clostridial Enterotoxicosis               48
Name: 진단명1, Length: 400, dtype: int64

In [119]:
encoder = LabelEncoder()
encoder.fit(df['진단명1'])
df['label_id'] = encoder.transform(df['진단명1'])

In [120]:
df.head()

,진단명,SE,len,진단명1,label_id
0,Anorexia,체온 호흡수 shivering 있음 mm p.pink skin turgor 약간 ...,337,Anorexia,8
1,Anorexia,cc 식욕부진 hpi 평소 식욕이 좋았으나 최근 열흘동안 식욕부진 계속 한달 전부터...,584,Anorexia,8
18,Splenic mass,cc 복강 내 종양 및 담낭 파열 의심 서울 예스ah hpi month 전 부터 연...,873,Splenic mass,200
19,Splenic mass,체온 호흡수 회 약간 끙끙거리면서 노력성 호흡 tlk time hr 정용량 cri...,436,Splenic mass,200
31,Food allergy,cc 소양감 hpi 보호자님 처자가 키우던 강아지 살 때 부터 보호자분이 키웠음 당...,677,Food allergy,80


In [122]:
train, test = train_test_split(df,test_size=0.1, random_state=100 )
train.to_csv('files/unlabel_train1.csv', index=None)
test.to_csv('files/unlabel_test1.csv', index=None)
df.to_csv('files/unlabel_df.csv', index=None)
print(train.shape)
print(test.shape)

(56187, 5)
(6244, 5)


In [123]:
df.head()

,진단명,SE,len,진단명1,label_id
0,Anorexia,체온 호흡수 shivering 있음 mm p.pink skin turgor 약간 ...,337,Anorexia,8
1,Anorexia,cc 식욕부진 hpi 평소 식욕이 좋았으나 최근 열흘동안 식욕부진 계속 한달 전부터...,584,Anorexia,8
18,Splenic mass,cc 복강 내 종양 및 담낭 파열 의심 서울 예스ah hpi month 전 부터 연...,873,Splenic mass,200
19,Splenic mass,체온 호흡수 회 약간 끙끙거리면서 노력성 호흡 tlk time hr 정용량 cri...,436,Splenic mass,200
31,Food allergy,cc 소양감 hpi 보호자님 처자가 키우던 강아지 살 때 부터 보호자분이 키웠음 당...,677,Food allergy,80


In [124]:
df.label_id.value_counts()

9      1571
219    1294
158    1184
176    1182
139    1009
       ... 
389      48
129      48
225      48
38       48
51       48
Name: label_id, Length: 400, dtype: int64

In [125]:
empty_df = pd.DataFrame(columns=('진단명', 'SE', 'len', '진단명1', 'label_id'))

In [130]:
for index, label in label_dict.items() :
    


Anti-inflamation
1571
dermatits-bacterial
1294
Pancreatitis
1184
Pulmonary Edema
1182
Malassezia Dermatitis
1009
Vomiting, Acute
991
Otitis Externa
968
RESPIRATORY DISTRESS SYNDROME, ACUTE
796
G-I trouble
638
췌장염(Pancreatitis)
626
불임수술 - 여
583
Dyspnea and Tachypnea
566
Chronic Renal Disease (CKD) - IRIS stage
554
Dermatophytosis
527
중성화수술 - 남
514
Cancer PS(Prevalence Survey)
514
스케일링,발치
482
슬개골탈구(Patellar Luxation)
481
자궁축농증(Pyometra)
475
Pyometra and Cystic Endometrial Hyperplasia
472
Blood Typing: DEA 1.1 positive
467
Cough
454
소염요법
451
슬개골 내측탈구 좌측(Lt) -G3
449
치석(Calculus)
448
방광결석(Urolithiasis)
437
스케일링
411
Patellar Luxation
402
ENTERITIS
393
Ear flushing
390
골절 - 단순골절(Fracture - Simple)
383
자궁축농증
381
Seizure(Unknown origin)
367
진통주사
366
Pneumonia, Bacterial
365
방광결석
356
Tracheal Collapse - Dogs
350
척추 추간판 질환(IVDD(InterVertebralDiskDisease))
346
Anemia
342
음식알러지-식품거부반응성 피부염(Food Reactions-Dermatologic)
341
Ear Mites
327
외이염(Otitis Externa)
315
Hypothyroidism(갑상선 기능저하증)
306
중성화수술 - 여